In [3]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

c:\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
#opening the file
path ='male.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower() #lowercasing each name to make it case insensitive
print('corpus length:', len(text)) #printing length of entire text

corpus length: 5201704


In [5]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 27


In [6]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 1733895


In [8]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [9]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [10]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    txt=""
    file = open("names.txt","a+") 
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(8):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
            file.write(sentence)
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print() 
    file.close()

In [12]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x, y,
          batch_size=128,
          epochs=2,
          callbacks=[print_callback])

Epoch 1/2
1733895/1733895 [==============================] - 468s 270us/step - loss: 1.7474

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "arshawn
dasan
dashay"
arshawn
dasan
dashayn
davis

----- diversity: 0.5
----- Generating with seed: "arshawn
dasan
dashay"
arshawn
dasan
dashay
davonta
----- diversity: 1.0
----- Generating with seed: "arshawn
dasan
dashay"
arshawn
dasan
dashay
navon
c
----- diversity: 1.2
----- Generating with seed: "arshawn
dasan
dashay"
arshawn
dasan
dashayn
daythu
Epoch 2/2
1733895/1733895 [==============================] - 523s 301us/step - loss: 1.4713

----- Generating text after Epoch: 1
----- diversity: 0.2
----- Generating with seed: "
maxwell
brice
haywo"

maxwell
brice
haywood
jayle
----- diversity: 0.5
----- Generating with seed: "
maxwell
brice
haywo"

maxwell
brice
haywood
justi
----- diversity: 1.0
----- Generating with seed: "
maxwell
brice
haywo"

maxwell
brice
haywood
joshy
----- diversity: 1.2
----- Genera